
## <span style="color:#F6A800"> I. Theoretical background </span>

In this exercise, the lowest energy eigenstates the helium atom is calculated using the variational Monte Carlo (VMC) method.

### <span style="color:#F6A800"> I.1 VMC method in quantum mechanics </span>

Remember the variational principle

$$ \langle \hat{H} \rangle = \frac{\langle{\psi_L} \hat{H} {\psi_L}\rangle }{\langle{\psi_L}{\psi_L}\rangle} =  \frac{ \int \psi_L^{*}(R) \hat{H} \psi_L(R) {\rm d}R}{\int \psi_L^{*}(R) \psi_L(R){\rm d}R} \geq E_0 $$

for the expectation value of $ \hat{H} $ with respect to a wave function $\psi_L$, where $ E_0 $ is the minmal energy of the system.

The relation above holds true exactly for a trial wave function that resembles the system in its lowest possible energy state $ E_0 $. As this function is generally unknown, except for some special cases, a trial wave function similar to the exact wave function in parametric form (_e.g._, which is variable by a parameter $ \alpha $ as in UE05) can be used.
By solving the equation above with different parameter values and searching for the minimum for $ \langle{H}\rangle $, a good estimation for $ E_0 $ can be obtained. The integral appearing in equation the above equation can be solved using Monte-Carlo integration.

### <span style="color:#F6A800"> I.2 Hamiltonian for atomic systems </span>

The Hamiltonian of an atomic system reads

$$ H(\mathbf{R}) = 
\left(-\frac{\mathbf{P}^2}{2M} - \sum_{j=1}^{N}\frac{\mathbf{p}_j^2}{2m} \right) 
+
\left(
- \sum_{i=1}^{N} \frac{Ze^2}{(4\pi \epsilon_0)r_i} + \sum_{i=1, i<j}^{N} \frac{e^2}{(4\pi\epsilon_0) r_{ij}}\right) $$



Furthermore, we apply the Born-Oppenheimer approximation.



Then, in electronic units ($ m = e = 4\pi\epsilon_0 = \hbar = 1  $), we have

$$ \boxed{
H(\mathbf{R}) = 
- \sum_{i=1}^{N}\frac{\mathbf{p}_i^2}{2}
- \sum_{i=1}^{N} \frac{Z}{r_i}
+ \sum_{i=1, i<j}^{N} \frac{1}{ r_{ij}}} $$

### <span style="color:#F6A800"> I.3 Trial wave function </span>

We expand the approach used in UE05 for the hydrogen atom to other simple atoms, which yields the trial wave function:

$$ \Psi_{L,\alpha}=e^{-\sum\limits_i \alpha \cdot Z \cdot r_i}=\prod e^{-\alpha \cdot Z \cdot r_i} $$

where $Z$ is the number of electrons -- which equals the number of protons as the atoms shall not be charged -- and $r_i$ the distance between the $i$-th electron and the nucleus.


### <span style="color:#F6A800"> I.4 Local energy </span>

By defining the **local energy** $ E_L$:

$$ E_L = \frac{1}{\psi_{L,\alpha}} H \psi_{L,\alpha} $$

the variational principle can be rewritten:

$$ \langle \hat{H} \rangle_\alpha 
= \frac{ \int \psi_{L,\alpha}^{*}(R)\psi_{L,\alpha}(R) E_L(R) {\rm d} R} {\int \psi_{L,\alpha}^{*}(R) \psi_{L,\alpha}(R){\rm d} R}
\geq E_0 $$

If $\Psi_{L,\alpha}$ is an energy eigenstate, the local energy $E_L$ does not depend on $R$, yielding the appoximation:

$$ \langle \hat{H} \rangle_\alpha \approx \dfrac{1}{M}\sum\limits_{i=0}^{M}E_L(r_i) $$

where $M$ is the number of Monte Carlo steps used in the calculation. The minimum of $\langle \hat{H} \rangle_\alpha$ -- which is overestimating the true ground state energy for all $\alpha$ -- provides a good approximation for the ground state energy $E_0$.

### <span style="color:#F6A800"> I.5 Implementation </span>

The variational Monte Carlo method follows the concept of first putting a chosen number of walkers at random electron positions $R_i$, then selecting the next walker and shifting it a chosen distance ${\rm d} r$ in random direction to the new positions $R_{\text{new},i}$. Next, the fraction

$$ W = \left(\dfrac{\Psi_{L,\alpha}(R_{\text{new},i})}{\Psi_{L,\alpha}(R_i)}\right)^2 $$

is calculated. If $W<1$, the new position is accepted with probability $W$ whereas if $W\geq 1$, the new position is accepted. During warm up, the shift distance ${\rm d} r$ is optimized such that during the calculation, in total around 50% of the new positions are accepted. In general MC algorithms, one usually aims for an acceptance ratio in the range of 20%-60%.

## <span style="color:#F6A800"> II. VMC for the helium atom </span>

Below, you can find a simple python implementation of the VMC method for the helium atom. The calculation might take a couple of minutes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 30 # number of steps to vary alpha
N = 5 # number of walkers
D = 3 # number of dimension
Z = 2 # number of electrons
dr = 2.0 # initial length of the Metropolis MC shift
alpha = np.linspace(0.2,1.6,A) # range to vary alpha
M = 10**5 # number of VMC steps
M1 = 25000 # number of steps for one adjustment of dr during thermalizing
L = 3 # number of adjustments of dr for each alpha during thermalizing
M2 = L*M1 # total number of steps to thermalize

def Psi_T(r1,r2): # trial wavefunction (r1, r2 are vectors)
    return np.exp(-Z*a*(r(r1)+r(r2))) # first trial function
def local_energy(r1,r2): # local energy (r1, r2 are vectors)
    return -(Z*a)**2+(Z*a)/r(r1)+(Z*a)/r(r2)-Z/r(r1)-Z/r(r2)+1/r(r1-r2) # for first trial function
def r(R): # gives the length of 3-dim vector R: |R|=r
    return np.sqrt(R[0]**2+R[1]**2+R[2]**2)
H = np.zeros((A,N)) # prepare for the expectation values <H>
for i in np.arange(A): # ***VARYING ALPHA***
    a = alpha[i]
    print("\n alpha =",a)
    E = np.zeros(N) # prepare to sum up local energies
    C = np.zeros(N) # prepare to count number of accepted positions
    for j in np.arange(N): # ***SELECTING WALKER***
        print("Walker No.", j)
        c = 0 # prepare to count number of accepted positions for thermalizing
        R1 = np.array([np.random.uniform(-1,1) for i in np.arange(D)]) # initial position for electron 1
        R2 = np.array([np.random.uniform(-1,1) for i in np.arange(D)]) # initial position for electron 2
        for n in np.arange(M2): # ***THERMALIZE POSITION OF ELECTRONS***
            dR1 = np.array([np.random.uniform(-10,10) for i in np.arange(D)]) # create vector dR1 to shift R1 to R1new
            dR1 = dr*dR1/r(dR1) # normalize dR1 to length dr: |dR1|=dr
            R1new = R1+dR1 # move the electron by dR1
            dR2 = np.array([np.random.uniform(-10,10) for i in np.arange(D)]) # create vector dR2 to shift R2 to R2new
            dR2 = dr*dR2/r(dR2) # normalize dR2 to length dr: |dR2|=dr
            R2new = R2+dR2 # move the electron by dR2
            W = (Psi_T(R1new,R2new)/Psi_T(R1,R2))**2 # calculate fraction p
            if W >= np.random.uniform(0,1): # condition to accept new position
                R1 = R1new
                R2 = R2new
                c += 1 # counter of number of accepted positions
            if (n+1)%(M1+1) == 0: # adjust length of Metropolis MC shift
                v = c/M1 # ratio accept/all, we aim for v=0.5
                dr *= v/0.5 # adjust dr
                c = 0
        for m in np.arange(M): # ***CALCULATE SUM OF LOCAL ENERGIES***
            dR1 = np.array([np.random.uniform(-10,10) for i in np.arange(D)]) # create vector dR1 to shift R1 to R1new
            dR1 = dr*dR1/r(dR1) # normalize dR1 to length dr: |dR1|=dr
            R1new = R1+dR1 # move the electron by dR1
            dR2 = np.array([np.random.uniform(-10,10) for i in np.arange(D)]) # create vector dR2 to shift R2 to R2new
            dR2 = dr*dR2/r(dR2) # normalize dR2 to length dr: |dR2|=dr
            R2new = R2+dR2 # move the electron by dR2
            W = (Psi_T(R1new,R2new)/Psi_T(R1,R2))**2 # calculate fraction p
            if W >= np.random.uniform(0,1): # condition to accept new position
                E[j] += local_energy(R1new,R2new) # sum up local energies
                C[j] += 1 # counter of number of accepted positions
                R1 = R1new
                R2 = R2new
            else:
                E[j] += local_energy(R1,R2)
        H[i,j]=E[j]/M # calculate expectation value <H> = average of local energies
        V = C[j]/M # ratio accept/all, we aim for V=0.5
        print("ratio accept/all = ", V)
Hsum = np.sum(H,axis=1)/N # mean over <H> for all alpha (for minimum: np.amin(H,axis=1))
Hmin = np.amin(H,axis=1)

<br>

Let's have a look at the result:

In [ ]:
print("E_0 =",np.min(H),"Ha") # calculate minimum of <H> = ground state energy E_0
print("E_0 =",np.min(H)*27.21138602,"eV") # unit converting: [E_0]=Hartree --> [E_0]=eV

In [ ]:
plt.figure(dpi=600)
plt.subplot(121)
for k in np.arange(N): # for each walker plot <H> vs. alpha
    plt.plot(alpha,H[:,k]*27.21138602, label="walker {}".format(k))
plt.xlabel(r"$\alpha$")
plt.ylabel(r"<H>=<$\Psi$|H|$\Psi$> (eV)")
plt.plot(alpha,-79.014245*np.ones(len(alpha)), label=r"literature value")
plt.legend(fontsize=8, loc=2)
plt.subplot(122)
plt.plot(alpha,Hsum*27.21138602, label="mean of walkers")
plt.plot(alpha,Hmin*27.21138602, label="minimum of walkers")
plt.plot(alpha,-79.014245*np.ones(len(alpha)), label=r"literature value")
plt.xlabel(r"$\alpha$")
plt.legend(fontsize=8, loc=2)
plt.show()